#TP3 : Decision tree

In [ ]:
import numpy as np
import math
import copy

In [ ]:
#Question 1a

#We create a fonction to extract data from the file
def dataFromFile(f):
    dataFile = open(f,'r')
    lines = dataFile.readlines()
    records = []
    attributes = [[],[],[]]
    types = lines[0].split(',')
    mot = types[-1].strip('\n')
    types[-1] = mot
    for line in lines[1:]:
        current = []
        cutLine = line.split(',')
        attributes[0].append(int(cutLine[0]))
        attributes[1].append(int(cutLine[1]))
        attributes[2].append(int(cutLine[2]))
        for i in cutLine:
            current.append(int(i))
        records.append(current)
    for i in range(0,3):
        dicattribute = dict.fromkeys(attributes[i])
        attributes[i] = list(dicattribute)
        attributes[i] = sorted(attributes[i])
    records = np.array(records)
    dataFile.close()
    #It returns all the data, the different possibilities for each type of data
    #and the names associated with each column
    return records, attributes, types

In [ ]:
#This function return the gini of a list of 0 or 1
def gini(group):
    total = len(group)
    if total == 0:
      g = 0.5
    else:
      C1=sum(group)
      C2=total - C1
      g = 1 - (C1/total)**2 - (C2/total)**2
    return g

In [ ]:
#This function returns the gini of a set of data with four columns
def gini2(D):
  group = []
  for line in D:
    group.append(line[3])
  return gini(group)

In [ ]:
#This function split the data in to lists depending on whether or not
#their value in the columnNb column is > splitValue
def splitData(records,columnNb,splitValue):
    recordSplit = [[],[]]
    for line in records:
        if line[columnNb]<=splitValue:
            recordSplit[0].append(line)
        else:
            recordSplit[1].append(line)
    return recordSplit

In [ ]:
#Determines the best split value for the column columnNb according to the gini
def giniSplit(records, attributes, columnNb):
    dataToSplit = [[],[]]
    possibleValues = attributes[columnNb]
    recordSplit = [[],[]]
    splitValue = 0
    minSp = 0.5

    for line in records:
        dataToSplit[0].append(line[columnNb])
        dataToSplit[1].append(line[3]) #survived or not: 1 or 0

    
    #if there are more than two possible values for the attribute
    if len(possibleValues)>2:
        giniSplitList = []
        for i in range(len(possibleValues)-1):
            res1 = []
            res2 = []
            for j in range(len(dataToSplit[0])):
                if  dataToSplit[0][j] <= possibleValues[i]:
                    res1.append(dataToSplit[1][j])
                else:
                    res2.append(dataToSplit[1][j])
                

            if len(res1)!=0 and len(res2)!=0:
              gini1 = gini(res1)
              gini2 = gini(res2)
              giniSp = (gini1*len(res1)/(len(res2)+len(res1)))+(gini2*len(res2)/(len(res2)+len(res1)))
              giniSplitList.append(giniSp)
            
            else:
              giniSplitList.append(0.5)


        else:
          minSp = min(giniSplitList)
          indexValue = giniSplitList.index(minSp)
          splitValue = possibleValues[indexValue]
          recordSplit = splitData(records,columnNb,splitValue)
    
    #if there are only two values for the attribute
    else:
        splitValue = possibleValues[0]
        recordSplit = splitData(records,columnNb,splitValue)
        res1 = []
        res2 = []
        for j in range(len(dataToSplit[0])):
          if  dataToSplit[0][j] <= splitValue:
            res1.append(dataToSplit[1][j])
          else:
            res2.append(dataToSplit[1][j])
        if len(res1)!=0 and len(res2) !=0:
          gini1 = gini(res1)
          gini2 = gini(res2)
          minSp = (gini1*len(res1)/(len(res2)+len(res1)))+(gini2*len(res2)/(len(res2)+len(res1)))
        else:
          minSp = 0.5
        
        
    return minSp, recordSplit, splitValue

In [ ]:
#From a file, we import data and launch the recursive function
def BuildDecisionTree(f, minNum):
  D,  A, names = dataFromFile(f)
  return linTree(Build(D,A, minNum,names,0))

In [ ]:
#Recursive fonction to create the DecisionTree
def Build(D, A, minNum,names,d):
  g = gini2(D)

  classBool = D[0][3]
  sameBool = True
  s = 0
  for i in D:
    if i[3] != classBool:
      sameBool = False
    s += i[3]
  if sameBool == True : #if they all survive or all die 
    return [0, classBool, D]

  elif len(D) < minNum :
    return [g,d,D]



  else : 
    
    #then we calculate which split is the best
    minSplit = 0.5
    minIndex = 0
    for i in range(len(A)):
      minSp, recordSplit, splitValue = giniSplit(D,A,i)
      if minSp <= minSplit:
        minSplit = minSp
        minIndex = i
        minSplitValue = splitValue
        D1 = recordSplit[0]
        D2 = recordSplit[1]
    if D1 == [] or D2 == []:
      cL = 0
      #we find the class by majority
      #Counting the occurences of 1 and 0:
      S = sum(D[i][3] for i in range(len(D)))/len(D)
       #There is a tie between the default and the majority class => default class
       #otherwise it is determined by majority.
      if S<=0.5:
        cL=0
      else:
        cL=1
      return [g,cL,D]
          
    else:
    #recursive call and add information (gini and feature)
      return [g, names[minIndex] +'<'+ str(minSplitValue+1), Build(D1, A, minNum,names,d), Build(D2, A, minNum,names,d)]

In [ ]:
#Recursively linearizing the tree we had
def linTree(decisionTree):
  T = []
  for i in range(2**6 -1):
    T.append([])
  lin(decisionTree, 0, T) #lauching recursion
  return T

In [ ]:
#Recursive function to linearize the tree
def lin(tree, position, T):
  T[position].append(tree[0])
  T[position].append(tree[1])
  if tree[1] == 0 or tree[1] == 1 : #it is a leaf
    T[position].append(tree[2])
  else: #it isn't a leaf so we have to browse deeper
    lin(tree[2], 2*position+1, T)
    lin(tree[3], 2*position+2, T)

In [ ]:
#Test
print(dataFromFile('sample_data/data.csv'))
dT = BuildDecisionTree('sample_data/data.csv',5)
print(dT)
print(dT[62])

(array([[1, 3, 2, 0],
       [0, 1, 0, 1],
       [0, 3, 2, 1],
       ...,
       [0, 3, 2, 0],
       [1, 1, 0, 1],
       [1, 3, 1, 0]]), [[0, 1], [1, 2, 3], [0, 1, 2]], ['Sex', 'Pclass', 'Embarked', 'Survived'])
[[0.4723650263627057, 'Sex<1'], [0.384430473372781, 'Pclass<3'], [0.3064437162277842, 'Pclass<2'], [0.10140306122448985, 'Embarked<2'], [0.5, 'Embarked<2'], [0.46560064498790643, 'Embarked<1'], [0.24174858108924058, 'Embarked<1'], [0.03702385190459241, 'Pclass<2'], [0.12945179584120986, 'Pclass<2'], [0.4228316326530613, 'Embarked<1'], [0.46875, 0, [array([0, 3, 2, 1]), array([0, 3, 2, 1]), array([0, 3, 2, 1]), array([0, 3, 2, 0]), array([0, 3, 2, 0]), array([0, 3, 2, 0]), array([0, 3, 2, 1]), array([0, 3, 2, 0]), array([0, 3, 2, 0]), array([0, 3, 2, 0]), array([0, 3, 2, 1]), array([0, 3, 2, 0]), array([0, 3, 2, 1]), array([0, 3, 2, 1]), array([0, 3, 2, 0]), array([0, 3, 2, 1]), array([0, 3, 2, 0]), array([0, 3, 2, 0]), array([0, 3, 2, 0]), array([0, 3, 2, 1]), array([0, 3, 

In [ ]:
#Question 1b
#The attribute is a recursive tree, it prints it nicely
def printDecisionTree(decisionTree):
  S = '' #initialistion of what we print
  depth = int(math.log2(len(decisionTree)+1)) #graph depth
  for i in range(depth):
    length = 2**i
    for j in range (length): #We browse every node every levels
      position = 2**i -1 +j
      if decisionTree[position]!= [] :
        #We label the type of node and add its level and feature
        if (position==0):
          S+= 'Root \n'
          S+= 'Level ' + str(i) + '\n'
          S+= 'Feature ' + str(decisionTree[position][1]) +'\n'
        elif (2*position >= len(decisionTree)-1):
          S+= 'Leaf \n'
          S+= 'Level ' + str(i) + '\n'
          S+= 'Class ' + str(decisionTree[position][1]) + '\n'
        elif (decisionTree[2*position +1]==[]):
          S+= 'Leaf \n'
          S+= 'Level ' + str(i) + '\n'
          S+= 'Class ' + str(decisionTree[position][1]) + '\n'
        else : 
          S+= 'Intermediate \n'
          S+= 'Level ' + str(i) + '\n'
          S+= 'Feature ' + str(decisionTree[position][1]) +'\n'
        #we add the gini   
        S+= 'Gini '+ str(decisionTree[position][0]) +'\n' 

        condition = False #determines if we haven't finish this level
        for k in range (j+1,length):
          if decisionTree[2**i-1+k]!= []:
            condition = True
        if (condition): #If we haven't
          S+= '*****\n'
        else :
          S += '\n'
  print (S)

In [ ]:
#Test 
printDecisionTree(dT)

Root 
Level 0
Feature Sex<1
Gini 0.4723650263627057

Intermediate 
Level 1
Feature Pclass<3
Gini 0.384430473372781
*****
Intermediate 
Level 1
Feature Pclass<2
Gini 0.3064437162277842

Intermediate 
Level 2
Feature Embarked<2
Gini 0.10140306122448985
*****
Intermediate 
Level 2
Feature Embarked<2
Gini 0.5
*****
Intermediate 
Level 2
Feature Embarked<1
Gini 0.46560064498790643
*****
Intermediate 
Level 2
Feature Embarked<1
Gini 0.24174858108924058

Intermediate 
Level 3
Feature Pclass<2
Gini 0.03702385190459241
*****
Intermediate 
Level 3
Feature Pclass<2
Gini 0.12945179584120986
*****
Intermediate 
Level 3
Feature Embarked<1
Gini 0.4228316326530613
*****
Leaf 
Level 3
Class 0
Gini 0.46875
*****
Leaf 
Level 3
Class 0
Gini 0.48185941043083896
*****
Intermediate 
Level 3
Feature Embarked<2
Gini 0.455
*****
Intermediate 
Level 3
Feature Pclass<3
Gini 0.3503025987896047
*****
Intermediate 
Level 3
Feature Embarked<2
Gini 0.22524194945669662

Intermediate 
Level 4
Feature Embarked<1
Gini 0.0

In [ ]:
#Question 2
#Browse the linearized tree to compute the generalization error
def generalizationError(dT, alpha):
  complexity = 0
  error = 0
  total = 0 
  for i in range(len(dT)): 
    if len(dT[i])==3: #it is a leaf
      complexity += 1 
      D = dT[i][2]
      total += len(D)
      survive = dT[i][1] #the class
      for i in D:
        if i[3]!= survive: #the class is different, it is an error
          error+=1
  return (error + alpha*complexity)

In [ ]:
generalizationError(dT,0.5)

176.5

In [ ]:
# question 3

def joinLeaf(PruneTree,n,minNum):

  if len(PruneTree[n*2+1]) == 2:
    joinLeaf(PruneTree,n*2+1,minNum)
  
  if len(PruneTree[n*2+2]) == 2:
    joinLeaf(PruneTree,n*2+2,minNum)

  if len(PruneTree[n*2+1]) == 3 and len(PruneTree[n*2+2]) == 3:
    leaf1, leaf2 = PruneTree[n*2+1], PruneTree[n*2+2]
    newArray = np.concatenate((leaf1[-1],leaf2[-1]))
    PruneTree[n*2+1], PruneTree[n*2+2]=[], []
    PruneTree[n].append(newArray)

    #Its class value is c if all records have class c
    classbool = newArray[0][3]
    samebool = True
    for array in newArray:
      if array[3] != classbool:
        samebool = False
    if samebool:
      PruneTree[n][1] = classbool
      
      #The default class if the number of records is < minNum (default class: 0)
    elif len(newArray)<minNum:
      PruneTree[n][1] = 0
      
      #Counting the occurences of 1 and 0:
    else:
      S = sum(newArray[i][3] for i  in range(len(newArray)))/len(newArray)

      #There is a tie between the default and the majority class => default class
      #otherwise it is determined by majority.
      if S<=0.5:
        PruneTree[n][1]=0
      else:
        PruneTree[n][1]=1

      #We calculate the new gini
    PruneTree[n][0]=gini2(newArray)

In [ ]:
def pruneTree(DecisionTree,alpha,minNum):
  PruneTree = copy.deepcopy(DecisionTree)
  FinalTree = copy.deepcopy(DecisionTree)
  minGE = generalizationError(FinalTree,alpha)
  for i in range(len(DecisionTree)-1,0,-1):
    #if it's not a leaf and it's a intermediate
    if len(PruneTree[i]) == 2:
      joinLeaf(PruneTree,i,minNum)
    if generalizationError(PruneTree,alpha) <= minGE:
      FinalTree = copy.deepcopy(PruneTree)
      minGE = generalizationError(PruneTree,alpha)
    PruneTree = copy.deepcopy(FinalTree)
  return FinalTree

In [ ]:
pT = pruneTree(dT,0.5,5)
printDecisionTree(pT)

Root 
Level 0
Feature Sex<1
Gini 0.4723650263627057

Intermediate 
Level 1
Feature Pclass<3
Gini 0.384430473372781
*****
Leaf 
Level 1
Class 0
Gini 0.3064437162277842

Leaf 
Level 2
Class 1
Gini 0.10140306122448985
*****
Intermediate 
Level 2
Feature Embarked<2
Gini 0.5

Leaf 
Level 3
Class 1
Gini 0.4228316326530613
*****
Leaf 
Level 3
Class 0
Gini 0.46875




In [ ]:
generalizationError(pT,0.5)

170.0